In [1]:
# !pip install httpx
import matplotlib
matplotlib.rcParams['font.sans-serif'] = "KaiTi"

## 网易接口

网易接口获取历史每日数据

URL 格式：


```
http://quotes.money.163.com/service/chddata.html?code=代码&start=开始时间&end=结束时间&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP
```


参数说明：代码为股票代码，上海股票前加0，如600756变成0600756，深圳股票前加1。时间都是6位标识法，年月日，fields标识想要请求的数据。可以不变。

例如大盘指数数据查询（上证指数000001前加0，沪深300指数000300股票前加0，深证成指399001前加1，中小板指399005前加1，创业板指399006前加1）：

```
http://quotes.money.163.com/service/chddata.html?code=0000300&start=20151219&end=20171108&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;VOTURNOVER
```


In [13]:
import httpx
import pandas as pd
import dtale
from io import StringIO 

url = "http://quotes.money.163.com/service/chddata.html?code=0000001&start=20220101&end=20220723&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP"
r = httpx.get(url)
r.encoding = 'gb18030'
data = pd.read_csv(StringIO(r.text), index_col=0, parse_dates=[0])
data

,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,换手率,成交量,成交金额,总市值,流通市值
日期,,,,,,,,,,,,,,
2022-07-22,'000001,上证指数,3269.9739,3293.6439,3246.7313,3276.1966,3272.0012,-2.0273,-0.0620,NaN,300690066,3.835760e+11,NaN,NaN
2022-07-21,'000001,上证指数,3272.0012,3300.5345,3272.0012,3297.6614,3304.7243,-32.7231,-0.9902,NaN,332358707,4.206966e+11,NaN,NaN
2022-07-20,'000001,上证指数,3304.7243,3308.3472,3287.7441,3291.5507,3279.4313,25.2930,0.7713,NaN,301515123,3.834227e+11,NaN,NaN
2022-07-19,'000001,上证指数,3279.4313,3283.9295,3256.0152,3278.7221,3278.1033,1.3280,0.0405,NaN,315651918,3.960424e+11,NaN,NaN
2022-07-18,'000001,上证指数,3278.1033,3278.4715,3226.2315,3235.0881,3228.0609,50.0424,1.5502,NaN,348346022,4.233573e+11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-10,'000001,上证指数,3593.5187,3593.5187,3555.1255,3572.7434,3579.5427,13.9760,0.3904,NaN,356222610,4.434008e+11,NaN,NaN
2022-01-07,'000001,上证指数,3579.5427,3607.2316,3577.0976,3588.9851,3586.0792,-6.5365,-0.1823,NaN,436306961,5.028914e+11,NaN,NaN
2022-01-06,'000001,上证指数,3586.0792,3594.4890,3559.8803,3581.2184,3595.1761,-9.0969,-0.2530,NaN,371540543,4.742843e+11,NaN,NaN


## Souhu 数据接口

搜狐数据接口：

http://q.stock.sohu.com/hisHq?code={code}&start={start}&end={end}，{code}替换为股票代码，大陆股票代码加前缀cn_。{start}替换为起始日期，{end}替换为结束日期。


例如：在浏览器地址栏输入：

http://q.stock.sohu.com/hisHq?code=cn_601766,cn_000002&start=20150601&end=20160414

数组中数据一次对应含义如下：

日期,
今日开盘价,
今日收盘价,
涨跌幅度，
涨跌百分比,
今日最低价,
今日最高价，
今日成交股票手数,
今日成交金额(单位：万)，
换手率

In [12]:
import json
url = "https://q.stock.sohu.com/hisHq?code=cn_600036&start=20220101&end=20220723"
r = httpx.get(url)
data = json.loads(r.text)
stock = data[0]['hq']
df = pd.read_json(json.dumps(stock), orient="records")
df.columns = ['日期', '开盘价','收盘价','涨跌幅度', '涨跌百分比','最低价', '最高价', '成交股票手数', '成交金额(单位：万)', '换手率']

df = df.set_index("日期")
df.index = pd.to_datetime(df.index)
df

,开盘价,收盘价,涨跌幅度,涨跌百分比,最低价,最高价,成交股票手数,成交金额(单位：万),换手率
日期,,,,,,,,,
2022-07-22,35.55,35.59,0.13,0.37%,35.25,36.05,510333,181747.73,0.25%
2022-07-21,35.54,35.46,-0.39,-1.09%,35.37,35.83,466241,165613.77,0.23%
2022-07-20,35.94,35.85,0.14,0.39%,35.71,36.12,469689,168511.69,0.23%
2022-07-19,35.50,35.71,0.08,0.22%,35.20,35.85,537814,191044.84,0.26%
2022-07-18,35.15,35.63,0.76,2.18%,35.01,35.84,903080,320222.72,0.44%
...,...,...,...,...,...,...,...,...,...
2022-01-10,49.47,49.23,0.18,0.37%,48.91,50.00,570936,281814.50,0.28%
2022-01-07,48.14,49.05,0.90,1.87%,48.04,49.14,660737,321952.34,0.32%
2022-01-06,48.82,48.15,-0.90,-1.83%,48.10,49.39,681867,329982.56,0.33%


## 新浪财经基金净值数据

url：

http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20220101&dateto=20220723&page=1

接口是分页的形式，需要遍历 page。

```
'日期' => 'fbrq',
'单位净值（元）' => 'jjjz',
'累计净值（元）' => 'ljjz',
```

In [10]:
page = 1
total = pd.DataFrame([])

while True:
    url = f'http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page={page}'
    #print(f'fetch url: {url}')
    r = httpx.get(url)
    data = json.loads(r.text)
    result = data["result"]["data"]['data']
    if (len(result) == 0):
        break
    df = pd.read_json(json.dumps(result), orient="records")
    total = pd.concat([total, df])
    page = page + 1


total.columns = ['日期', '单位净值（元）', '累计净值（元）']
total = total.set_index('日期')
total.index = pd.to_datetime(total.index)
total

,单位净值（元）,累计净值（元）
日期,,
2022-07-22,2.7210,4.2510
2022-07-21,2.7123,4.2423
2022-07-20,2.7432,4.2732
2022-07-19,2.7116,4.2416
2022-07-18,2.7372,4.2672
...,...,...
2021-01-08,2.8864,4.4164
2021-01-07,2.9176,4.4476
2021-01-06,2.8316,4.3616


## 新浪财经

新浪财经网页地址：https://vip.stock.finance.sina.com.cn/mkt/#hqIndex

url地址：

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=80&sort=symbol&asc=1&node=hs_a&symbol=&_s_r_a=init
```

`node` 参数：hs_a 为沪深A股


In [22]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=80&sort=symbol&asc=1&node=hs_a&symbol=&_s_r_a=init"
r = httpx.get(url)
data = json.loads(r.text)
df = pd.read_json(json.dumps(data), orient="records")
df = df.set_index("symbol")
df

,code,name,trade,pricechange,changepercent,buy,sell,settlement,open,high,low,volume,amount,ticktime,per,pb,mktcap,nmc,turnoverratio
symbol,,,,,,,,,,,,,,,,,,,
bj430047,430047,诺思兰德,11.05,-0.18,-1.603,11.04,11.05,11.23,11.33,11.37,11.05,255368,2851591,15:30:17,-55.250,12.201,284778.614315,164931.690040,0.17109
bj430090,430090,同辉信息,3.52,0.04,1.149,3.51,3.52,3.48,3.49,3.70,3.39,3895111,13870589,15:30:17,18.664,2.108,70165.408192,42517.603744,3.22473
bj430198,430198,微创光电,5.27,0.02,0.381,5.22,5.25,5.25,5.28,5.28,5.21,80019,419949,15:30:17,15.500,1.291,65414.431512,34033.043937,0.12391
bj430418,430418,苏轴股份,8.74,0.00,0.000,8.74,8.76,8.74,8.78,8.86,8.68,100983,889302,15:30:17,11.205,1.476,84533.280000,44763.020854,0.19717
bj430489,430489,佳先股份,6.35,-0.30,-4.511,6.34,6.35,6.65,6.63,6.63,6.35,462222,2975358,15:30:17,8.699,1.754,86651.592000,53203.747415,0.55167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bj836892,836892,广咨国际,9.36,0.06,0.645,9.35,9.36,9.30,9.36,9.36,9.34,3000,28057,15:30:17,9.649,2.722,92918.920536,23487.165000,0.01196
bj837092,837092,汉鑫科技,16.02,-0.10,-0.620,15.97,15.98,16.12,16.23,16.23,15.89,70362,1123847,15:30:17,13.024,1.949,76729.392000,20539.128258,0.54881
bj837212,837212,智新电子,8.47,-0.01,-0.118,8.40,8.47,8.48,8.51,8.58,8.39,120166,1016391,15:30:17,14.356,2.342,89866.700000,44129.970500,0.23064


**基金数据：**

url：

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=2&num=40&sort=symbol&asc=1&node=open_fund&_s_r_a=page
```


In [41]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=1&num=80&sort=symbol&asc=1&node=open_fund&_s_r_a=page"
r = httpx.get(url)
data = json.loads(r.text)
df = pd.read_json(json.dumps(data), orient="records", dtype={'symbol': str})
df.columns = ["代码", "名称", "最新公布净值", "累计净值", "前单位净值", "净值增长率", "日期", "基金规模份额（亿份）"]
df = df.set_index('代码')
df

,名称,最新公布净值,累计净值,前单位净值,净值增长率,日期,基金规模份额（亿份）
代码,,,,,,,
000001,华夏成长混合,1.0960,3.6570,1.1160,-1.792115,2022-08-12,31.1980
000003,中海可转债债券A,0.8960,1.1060,0.8970,-0.111483,2022-08-12,0.2712
000004,中海可转债债券C,0.8870,1.0970,0.8880,-0.112613,2022-08-12,0.3357
000005,嘉实增强信用定期债券,1.0409,1.4605,1.0412,-0.028813,2022-08-12,1.1954
000006,西部利得量化成长混合A,2.1718,2.4998,2.2009,-1.322186,2022-08-12,7.5167
...,...,...,...,...,...,...,...
000112,易方达纯债1年定期开放债券C,1.0630,1.4440,1.0630,0.000000,2022-08-12,0.1079
000116,嘉实丰益纯债定期债券,1.0158,1.4065,1.0175,-0.167076,2022-08-12,6.2611
000117,广发轮动配置混合,2.6720,2.6720,2.6670,0.187477,2022-08-12,1.6352


**ETF数据：**

url:

```
https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=3&num=40&sort=symbol&asc=1&node=etf_jz_fund&_s_r_a=page
```

In [45]:
url = "https://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getFundNetData?page=1&num=80&sort=symbol&asc=1&node=etf_jz_fund&_s_r_a=page"
r = httpx.get(url)
data = json.loads(r.text)
df = pd.read_json(json.dumps(data), orient="records", dtype={'symbol': str})
df.columns = ["代码", "名称", "最新公布净值", "累计净值", "前单位净值", "净值增长率", "日期", "基金规模份额（亿份）"]
df = df.set_index('代码')
df

,名称,最新公布净值,累计净值,前单位净值,净值增长率,日期,基金规模份额（亿份）
代码,,,,,,,
159001,易方达保证金货币A,0.4448,1.5530,0.0000,NaN,2022-08-12,0.0239
159003,招商保证金快线货币A,0.4095,1.4970,0.0000,NaN,2022-08-12,0.0183
159005,汇添富收益快钱货币A,0.4413,1.6070,0.0000,NaN,2022-08-12,0.0024
159601,华夏MSCI中国A50互联互通ETF,0.8724,0.8724,0.8705,0.218265,2022-08-12,64.6859
159602,南方MSCI中国A50互联互通ETF,0.8719,0.8719,0.8704,0.172335,2022-08-12,33.4854
...,...,...,...,...,...,...,...
159741,嘉实恒生科技ETF(QDII),0.5561,0.5561,0.5519,0.761007,2022-08-12,5.7080
159742,博时恒生科技ETF(QDII),0.5674,0.5674,0.5631,0.763630,2022-08-12,8.6993
159743,博时湖北新旧动能转换ETF,0.8990,0.8990,0.9071,-0.892956,2022-08-12,3.3194
